# Creating EEG Objects

## Epoch Creation
<a id="intro"></a>

In [ ]:
from simpl_eeg import eeg_objects

<br>

### Module Overview

The `eeg_objects` module contains helper classes for storing and manipulating relevant information regarding epochs to pass to other package functions. It contains two classes. Typically you will only you use the `eeg_objects.Epochs` directly, which by default contains a `eeg_objects.EEG_File` object in the `eeg_file` attribute. 
Below are the docstrings for the two classes:

In [ ]:
# Class for reading and importing EEG files
help(eeg_objects.EEG_File)

In [ ]:
# Class for storing, generating, and adjusting epoch objects
help(eeg_objects.Epochs)

<br>

### Define parameters

The only required parameter to create an epoch object is the `folder_path` for the experiment of interest, however additional parameters may be used to customize your epoch object. 

- `file_name`
    - If you specify a `file_name`, and the file exists in the `folder_path` directory, then it will be used as the main data file for the epoch.
    - If you do not specify a `file_name` then the alphabetical first file with a supported main file type in `folder_path` will be automatically loaded.


- `events_file`
    - If you specify an `events_file`, and the file exists in the `folder_path` directory, then it will be used as the events data file for the epoch.
    - If you do not specify an `events_file` then the alphabetical first file with a supported events file type in `folder_path` will be automatically loaded.
    - If you try to load an `events_file` (automatically or manually) with over 5,000 events or if the final column in the loaded dictionary does not contain a numerical value in its first index (both forms of error catching) then the file will be rejected and will not be loaded.
    - If you want to force no events data to be loaded you can pass and `events_file` of `None`.
   

- `montage`
    - If you specify a `montage`, it will load a standard montage with the specified name into the epoch data.
    - If montage data already exists in the main data file and a `montage` is provided the original data overwritten in the epoch object.
    - If you do not specify a `montage` and montage data already exists in the main data then it will be used instead.
    - If you do not specify a `montage` and montage data does not exist in the main data then one attempt will be made to load a "easycap-M1" montage. If this fails then no montage information will be loaded.
    - If you want to force no `montage` to be loaded data to be loaded you can pass and `events_file` of `None`.


- `start_second`
    - If you specify a `start_second`, a single epoch will be generated with an impact event at the specified second.
    - If you do not specify a `start_second`, epochs will be automatically generated using the impact times found in the `impact locations.mat` file in the selected `experiment_folder`. 


- `tmin`
    - specifies the number of seconds before the impact to use,


- `tmax`
    - specifies the number of seconds after the impact.

In [ ]:
# path to the experiment folder
folder_path = "../../data/109"

# the name of the main data file to load (optional)
file_name = "fixica.set"

# the name of the events file to load (optional)
events_file = "impact locations.mat"

# the montage type to load (optional)
montage = None

# number of seconds before the impact, should be a negative number for before impact (optional)
tmin = -1

# number of seconds after the impact (optional)
tmax = 1

# if creating a custom epoch, select a starting second (optional)
start_second = None

<br>

### Create epoched data

The following data formats are currently supported. Note that due to limited availability of test files not all formats have been fully tested (see Notes).

|                       | Main File | Secondary File | Events File | Notes                                                   |
|-----------------------|-----------|----------------|-------------|---------------------------------------------------------|
| EEGLAB                | .set      | .fdt           | .mat        |                                                         |
| BrainVision           | .vhdr     | .eeg           | .vmrk       |                                                         |
| European data format  | .edf      | N/A            | N/A         |                                                         |
| BioSemi data format   | .bdf      | N/A            | N/A         | Montage has not be successfully loaded with test files. |
| General data format   | .gdf      | N/A            | N/A         | Events have not be successfully loaded with test files. |
| Neuroscan CNT         | .cnt      | N/A            | N/A         | Montage has not be successfully loaded with test files. |
| eXimia                | .nxe      | N/A            | N/A         | Events have not be successfully loaded with test files. |
| Nihon Kohden EEG data | .eeg      | .pnt AND .21e  | .log        | Montage has not be successfully loaded with test files. |

- A **main file** represents the lead file used to load in your EEG data. This is the file that may be passed as your `file_name`.

- A **secondary file** contains some secondary information for some data types. They will be automatically loaded to when the main file is loaded.

- A **events file** contains a list of the annotations associated with events in your EEG data. This is the file that may be passed as your `events_file`.

- A **montage** must exist in your epoch in order to visualize it. This contains information about your node locations in 3D space. A complete list of usable montages is available here: https://mne.tools/dev/generated/mne.channels.make_standard_montage.html.

You can create epoched data using the `Epochs` class.

In [ ]:
epochs = eeg_objects.Epochs(
    folder_path = folder_path,
    file_name = file_name,
    events_file = events_file,
    montage = montage,
    tmin = tmin,
    tmax = tmax,
    start_second = start_second
)

The generated epoch data is found within the `all_epochs` attribute. Here we are generating epochs with automatically detected impact times, so we can see that there are multiple events.

In [ ]:
epochs.all_epochs

If instead we create epochs with a custom start second, we will only create a single epoch with an impact the given `start_second`.

In [ ]:
start_second = 15  # record event at second 15
custom_epoch = eeg_objects.Epochs(folder_path, tmin, tmax, start_second) 

custom_epoch.all_epochs

#### Get information about epochs

In addition to the epochs contained in the `all_epochs` attribute, the `Epoch` object also contains information about the file used and has a selected epoch for quick access. 

In [ ]:
eeg_file = epochs.eeg_file
print(eeg_file.folder_path)  # experiment folder path
print(eeg_file.experiment)  # experiment number
print(eeg_file.raw)  # raw data
print(eeg_file.file_source) # primary data file the EEG data was loaded from
print(eeg_file.events_source) # source file of events
print(eeg_file.montage_source) # source of the montage (may be pre-set montage name)
print(eeg_file.events)  # impact times

#### Select specific epoch

If you have a specific epoch of interest you can specify it with the `get_epoch` method. You can retrieve it later by accessing the `epoch` attribute.

In [ ]:
nth_epoch = 5  # the epoch of interest to select, the 6th impact
single_epoch = epochs.get_epoch(nth_epoch)
single_epoch

In [ ]:
epochs.epoch

#### Getting an evoked object

You can also use the `get_epoch` method to retrieve an evoked object, which represents an averaging of each event in your epoch. Note that evoked data is its own type of object and is not guaranteed to work with every function in this package.

In [ ]:
evoked = epochs.get_epoch("evoked")
type(evoked)

In [ ]:
evoked.info

#### Decimate the epoch (optional)
To reduce the size of the selected epoch you can choose to skip a selected number of time steps by calling the `skip_n_steps` method. If `use_single=True` (the default), it will only be run on the current selected epoch from the previous step, contained in the `epoch` attribute. Otherwise it will run on all the epochs contained within the `all_epochs` attribute.

Skipping steps will greatly reduce animation times for the other functions in the package. The greater the number of steps skipped, the fewer the frames to animate. In the example below we are reducing the selected epoch from 4097 time steps to 81 time steps. 

In [ ]:
single_epoch.get_data().shape

In [ ]:
num_steps = 50
smaller_epoch = epochs.skip_n_steps(num_steps)
smaller_epoch.get_data().shape

#### Average the epoch (optional)
To reduce the size of the selected epoch you can choose to average a selected number of time steps by calling the `average_n_steps` method. It will be run on the current selected epoch from the previous step, contained in the `epoch` attribute.

Averaging works the same way as decimating above, but instead of simply ignoring records between steps it takes an average. 

In [ ]:
num_steps = 50
average_epoch = epochs.average_n_steps(num_steps)
average_epoch.get_data()

### MNE functions

Now that you have access epoched data, you can use the `simpl_eeg` package functions as well as any [MNE functions](https://mne.tools/stable/generated/mne.Epochs.html) which act on `mne.epoch` objects. Below are some useful examples for the MNE objects contained within the object we created. 

#### Raw data
https://mne.tools/stable/generated/mne.io.Raw.html

In [ ]:
raw = epochs.eeg_file.raw
raw.info

In [ ]:
raw.plot_psd();

#### Epoch data

In [ ]:
# first 3 epochs
epochs.all_epochs.plot(n_epochs=3);

In [ ]:
# specific epoch
epochs.epoch.plot();

In [ ]:
# specific epoch with steps skipped
epochs.skip_n_steps(100).plot();